# Getting Started

**If you can read this you must have completed the CSIRO EASI Data Cube training environment on PC (easi-pc) installation. AWESOME!**

In this notebook we'll show you how to initialise and populate the sample data into your local install of the easi-pc. Almost the exact same approach can be used for any Open Data Cube installation but if you are using a hosted version (e.g. CSIRO Data Cube on AWS) data management will be controlled by the central authority and it will almost certainly provide other methods for user and shared data. For now though, you are the authority for your local installation.

## What we are about to do

1. Learn some tips about using the easi-pc notebooks
1. Initialise the database
2. Add an Earth Observation data product description to the database
3. Index some data in place without transformation
4. Ingest some data - make a copy of the data and transform it to a compute ready form to save on repeated calculations (e.g. reprojection, tiling, different file layout)

Along the way we will also learn some things about Docker and how to use it so you can save yourself from mistakes or save yourself some time. Keep an eye out for ___Docker tip:___. We'll also include ___Jupyter tip:___ and ___Play tip:___ along the way so you can have a better learning experience.

___Play tip:___ _The sample data is relatively small and its quite simple (and fast) to rebuild the easi-pc environment if you make a mistake or want to experiment with other data of your own and want to restart._


# Tips on using the easi-pc notebooks

___Jupyter tip:___ _You will see some common cells in all the training notebooks, particularly at the start. These usually setup notebook related environment information which impact how things are displayed. This next one starting with % tells jupyter we'd like all matplot lib graphics to be placed inline in the notebook, not in a separate window. We won't describe these over and over and of course an internet search will find most of these very easily._

In [2]:
%matplotlib inline

Here's another example of some common code. This time its straight Python (no special characters at the start). We use `pandas.DataFrame` objects to display our tables, so we will set some pandas settings to tweak their formatting so they look nice in the notebook.

In [3]:
import pandas
pandas.set_option('display.max_colwidth', 200)
pandas.set_option('display.max_rows', None)

One more example, by default python will display warnings in the output which display as red text in the output areas of the notebook. Most of these warnings are harmless unless you are developer (e.g. they are warning to let developers know a certain function is going to removed in the future and should be replaced by its new version). Whilst you can mostly just ignore the warning they can be repeated many times and clutter up the notebook display. Sometimes though things don't work and you want to turn the logging on so you can see what the error is and fix it.
Thankfully you can show and hide errors in the notebook. In the tutorial notebooks you will find the cell that follows placed at the end of the notebook. If you execute the cell it will create a link you can press to toggle the error display for the entire notebook. You only need to run the cell once but there is no harm in running it multiple times.

In [4]:
from IPython.display import HTML

HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_error').hide();
 $('div.output_stderr').hide();
 } else {
 $('div.output_error').show();
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
<form action="javascript:code_toggle_err()"><input type="submit" value="Click here to toggle on/off the error output."></form>''')


The cell below produces an error as the module isn't found. Execute the cell above so you can see the toggle button in the output then run the cell below. Click the Toggle button and you should see the error section show/hide.

In [5]:
import SomethingThatDoesntExistSoThereIsAnError

ModuleNotFoundError: No module named 'SomethingThatDoesntExistSoThereIsAnError'

___Play tip:___ _This error suppresion code has nothing to do with the easi-pc. You can copy and paste that cell into any notebook you might have and reuse it_

# Initialise the database

When you first install the ODC docker images the database is completely blank and requires:
1. An ODC database schema to be initialise
1. EO product information (metadata) to be added that describe the EO data attributes. There are multiple of these dependent on our data sources
1. An index of the actual EO data

The ODC contains a set of command line utilities for initialising the database. First lets check to see what state the database is in and if we can connect to it:

___Jupyter tip:___ _You can execute a command line program from a Jupyter cell by proceeding with the ! mark. To do this on an actual command line you would remove the ! mark._


In [8]:
!datacube system check

fatal: not a git repository: /home/jovyan/odc/../.git/modules/datacube-core
/home/jovyan/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Version:       0+unknown
Config files:  /home/jovyan/.datacube.conf
Host:          postgres:5432
Database:      odc
User:          odc
Environment:   None
Index Driver:  default

Valid connection:	YES


In [7]:
!datacube system init

fatal: not a git repository: /home/jovyan/odc/../.git/modules/datacube-core
/home/jovyan/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Initialising database...
Created.
Checking indexes/views.
Done.


# Add a product definition for Landsat data from USGS

In [10]:
!datacube product add ~/work/data-pipelines/landsat-usgs/ls8_usgs_sr_scene.yaml

fatal: not a git repository: /home/jovyan/odc/../.git/modules/datacube-core
/home/jovyan/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Added "ls8_usgs_sr_scene"


Verify the product definition loaded correctly. We'll look into what this code does later but for now you when it is run you should see a neat little table and the name of the product we just added. Then the next cell will display the measurements that it supports

In [11]:
# A jupyter magic to ensure out matploblib displays are inline in the notebook
%matplotlib inline
# Import pandas and set some parameters so the cells display nicely in our notebook
import pandas
pandas.set_option('display.max_colwidth', 200)
pandas.set_option('display.max_rows', None)

import datacube
dc = datacube.Datacube()
products = dc.list_products()

display_columns = ['name', 'description', 'platform', 'instrument', 'crs', 'resolution']

products[display_columns]

/home/jovyan/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


,name,description,platform,instrument,crs,resolution
id,,,,,,
1,ls8_usgs_sr_scene,Landsat 8 USGS Collection 1 Higher Level SR scene proessed using LaSRC. 30m UTM based projection.,LANDSAT_8,OLI_TIRS,NaN,NaN


In [12]:
# Get the measurements
measurements = dc.list_measurements()
# We can restrict which measurement attributes are displayed to reduce clutter
display_columns = ['units', 'nodata', 'aliases']
measurements[display_columns]

units  nodata  \
product           measurement                            
ls8_usgs_sr_scene coastal_aerosol  reflectance   -9999   
                  blue             reflectance   -9999   
                  green            reflectance   -9999   
                  red              reflectance   -9999   
                  nir              reflectance   -9999   
                  swir1            reflectance   -9999   
                  swir2            reflectance   -9999   
                  pixel_qa           bit_index       1   
                  sr_aerosol         bit_index       0   
                  radsat_qa          bit_index       1   

                                                                aliases  
product           measurement                                            
ls8_usgs_sr_scene coastal_aerosol                    [band_1, sr_band1]  
                  blue                               [band_2, sr_band2]  
                  green                              [band_3, sr_band3]  
                  red                                [band_4, sr_band4]  
                  nir                                [band_5, sr_band5]  
                  swir1                              [band_6, sr_band6]  
                  swir2                              [band_7, sr_band7]  
                  pixel_qa                                   [pixel_qa]  
                  sr_aerosol       [sr_aerosol_qa, aerosol_qa, aerosol]  
                  radsat_qa                                 [radsat_qa]

# Index some data

First, lets check to see if you have the data in the right place. If the data is already unpacked you should see a list of directories (each line begins with drwx...)



In [10]:
# TODO: Should we keep the tar file somewhere on make it part of the repo or ??
# TODO: May need to untar it in this cell if it is simply mounted

In [13]:
!ls -al /data/ls8_USGS_ESPA_data/


total 360
drwxrwxrwx 2 root root  81920 Nov 16 02:18 .
drwxrwxrwx 2 root root      0 Nov 15 23:12 ..
drwxrwxrwx 2 root root      0 Oct  9 01:37 LC080900842017090401T1-SC20180921064929
drwxrwxrwx 2 root root      0 Oct  9 01:37 LC080900842017092001T1-SC20180921064913
drwxrwxrwx 2 root root      0 Oct  9 01:38 LC080900842017100601T1-SC20180921064103
drwxrwxrwx 2 root root      0 Oct  9 01:38 LC080900842017102201T1-SC20180921063749
drwxrwxrwx 2 root root      0 Oct  9 01:38 LC080900842017110701T1-SC20180921070114
drwxrwxrwx 2 root root      0 Oct  9 01:38 LC080900842017112301T1-SC20180921063818
drwxrwxrwx 2 root root      0 Oct  9 01:38 LC080900842017120901T1-SC20180921063946
drwxrwxrwx 2 root root      0 Oct  9 01:39 LC080900842017122501T1-SC20180921065232
drwxrwxrwx 2 root root      0 Oct  9 01:39 LC080900842018011001T1-SC20180921063935
drwxrwxrwx 2 root root      0 Oct  9 01:39 LC080900842018012601T1-SC20180921083645
drwxrwxrwx 2 root root      0 Oct  9 01:39 LC080900842018021101T1-SC2

Now we run prepare script which will go through all the directories and their content gathering up all the metadata required for the datacube index and verifying everything is as it should be

In [14]:
!rm -f /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
!touch /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml && python3 ~/work/data-pipelines/landsat-usgs/easi_prepare_ls_usgs_sr.py --output /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml /data/ls8_USGS_ESPA_data/LC*/

2018-11-20 04:35:05,620 INFO Processing /data/ls8_USGS_ESPA_data/LC080900842017090401T1-SC20180921064929
2018-11-20 04:35:05,680 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:05,680 INFO Processing /data/ls8_USGS_ESPA_data/LC080900842017092001T1-SC20180921064913
2018-11-20 04:35:05,714 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:05,714 INFO Processing /data/ls8_USGS_ESPA_data/LC080900842017100601T1-SC20180921064103
2018-11-20 04:35:05,748 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:05,749 INFO Processing /data/ls8_USGS_ESPA_data/LC080900842017102201T1-SC20180921063749
2018-11-20 04:35:05,796 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:05,796 INFO Processing /data/ls8_USGS_ESPA_data/LC080900842017110701T1-SC20180921070114
2018-11-20 04:35:05,834 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:05,834 INFO Processing /data/ls8_USGS_ESPA_data/LC0809008420171

2018-11-20 04:35:07,498 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:07,498 INFO Processing /data/ls8_USGS_ESPA_data/LC080900852018080601T1-SC20180921063818
2018-11-20 04:35:07,539 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:07,539 INFO Processing /data/ls8_USGS_ESPA_data/LC080900852018082201T1-SC20180921063636
2018-11-20 04:35:07,584 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:07,585 INFO Processing /data/ls8_USGS_ESPA_data/LC080900852018090701T1-SC20180921063706
2018-11-20 04:35:07,623 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:07,623 INFO Processing /data/ls8_USGS_ESPA_data/LC080910842017091101T1-SC20180921064953
2018-11-20 04:35:07,666 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:07,667 INFO Processing /data/ls8_USGS_ESPA_data/LC080910842017092701T1-SC20180921082808
2018-11-20 04:35:07,710 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2

2018-11-20 04:35:09,369 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:09,370 INFO Processing /data/ls8_USGS_ESPA_data/LC080910852018071201T1-SC20180921064154
2018-11-20 04:35:09,410 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:09,410 INFO Processing /data/ls8_USGS_ESPA_data/LC080910852018072801T1-SC20180921064927
2018-11-20 04:35:09,449 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:09,450 INFO Processing /data/ls8_USGS_ESPA_data/LC080910852018081301T1-SC20180921063806
2018-11-20 04:35:09,486 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:09,486 INFO Processing /data/ls8_USGS_ESPA_data/LC080910852018082901T1-SC20180921064858
2018-11-20 04:35:09,527 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml
2018-11-20 04:35:09,527 INFO Processing /data/ls8_USGS_ESPA_data/LC080910852018091401RT-SC20180921063728
2018-11-20 04:35:09,572 INFO Writing /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml


In [15]:
!datacube dataset add /data/ls8_USGS_ESPA_data/ls8_usgs_sr.yaml

fatal: not a git repository: /home/jovyan/odc/../.git/modules/datacube-core
/home/jovyan/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Indexing datasets  [####################################]  100%


# Ingest some data